In [ ]:
!pip uninstall tensorflow --yes
!pip install tensorflow_decision_forests
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2 --yes
!pip install kerassurgeon

In [ ]:
import os
import cv2
import gc
import numpy as np
import pandas as pd
import glob
import pathlib
import tensorflow as tf
import tensorflow_addons as tfa
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import datasets, layers, models,Input,Model 
import tensorflow_datasets as tfds
from keras.models import Sequential
from keras.layers import  Bidirectional, Conv2D, BatchNormalization, MaxPooling2D, Flatten, LSTM, Dense, Lambda, Dropout,Reshape
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from tensorflow.keras.metrics import Accuracy, Recall,Precision
from sklearn.tree import DecisionTreeClassifier as Decisiontree
from sklearn.svm import SVC as Supportvectorclassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time
from functools import reduce
from kerassurgeon.operations import delete_layer, insert_layer
from keras.utils import to_categorical
import pickle

# Load data

In [ ]:
with open("../input/feature-extraction-only/features.pkl","rb") as file:
    features=pickle.load(file)
    
with open("../input/feature-extraction-only/features_test.pkl","rb") as file:
    features_test=pickle.load(file)
    
with open("../input/feature-extraction-only/lab_test.pkl","rb") as file:
    lab_test=pickle.load(file)
    
with open("../input/feature-extraction-only/labs.pkl","rb") as file:
    labs=pickle.load(file)

# Random Forest

Random forest is an ML technique that exploits a series of decision trees referred to as a forest. It is created from several decision trees aimed at developing a comprehensive and accurate stable forecast with a minimal internal biased estimate. Also, bootstrap aggression is used to train the random forest as every new tree is modified as per the bootstrap training observations sample. The ML technique entails both observable data and features to develop the most suitable prediction model from a given data. The model enables the connection of data via several links known as trees, where each tree subdivides its links into potential estimations from every data variable. The final model prediction is obtained from the average mean prediction from the available data. The Machine Learning model can utilize either mtree method that favor’s the features available in every tree, or ntree that favor’s the quantity of trees used.

Choosing parameters to try

In [ ]:
# tuning hyper parameter
parameters={"max_depth" : list(map(int,range(1,features.shape[0]-1,round((features.shape[0]-1)/5)))),
           "min_samples_leaf":[2,5,8,11],
           "min_weight_fraction_leaf":[0.1,0.2,0.3,0.5],
           "max_features":["auto","log2","sqrt",None],
           "max_leaf_nodes":[None,10,40,70],
           "random_state":[0]}

In [ ]:
parameters

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
st = time.time()
pca = PCA(n_components=1000)
pca.fit(features)
et = time.time()
round(st-et,4)

In [ ]:
exp_var = pca.explained_variance_ratio_ * 100
cum_exp_var = np.cumsum(exp_var)

In [ ]:
cum_exp_var

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(range(1,len(cum_exp_var)+1), cum_exp_var)
plt.title('Total variance explained')
plt.xlabel('Number of components ')
plt.ylabel('Explained variance')
plt.show()
plt.savefig('Explained.png')

In [ ]:
features = pca.fit_transform(features)
features_test = pca.transform(features_test)


In [ ]:
st = time.time()
RF = RandomForestClassifier(random_state = 0,n_jobs=-1)
RF=RF.fit(features,labs)
# Grid search
np.random.seed(100)
tuned=GridSearchCV(RF,param_grid=parameters,cv=3,n_jobs=-1)
tuned.fit(features,labs)
RF =RandomForestClassifier(**tuned.best_params_,n_jobs=-1)
RF =RF.fit(features,labs)
tuned.best_params_
et = time.time()
round((et - st))

In [ ]:
et = time.time()
pred=RF.predict(features_test)
et = time.time()
elapsed_time=round((et - st)/len(lab_test),4)
cm = metrics.confusion_matrix(lab_test, pred)
metrics.plot_confusion_matrix(RF,features_test,lab_test,display_labels=['COVID', 'Normal'],
                             cmap="YlOrBr",normalize='true')

In [ ]:
Randomf=pd.DataFrame({'Measure':['Accuracy','Sensitivity','Specitivity','Precision','F1_score','Excecution time'],
             'Random Forest':[metrics.accuracy_score(lab_test, pred),
                        metrics.recall_score(lab_test, pred,pos_label=0),
                        metrics.recall_score(lab_test, pred,pos_label=1),
                        metrics.precision_score(lab_test, pred,pos_label=0),
                        metrics.f1_score(lab_test, pred,pos_label=0),
                              elapsed_time]})
Randomf

In [ ]:
with open("RF.pkl","wb") as file:
    pickle.dump(RF,file)
    
#with open("original_hist.pkl","rb") as file:
#    hist=pickle.load(file)